# GPT-3.5-Turbo Model
Creating a question answering chatbot using GPT-3.5. Adapted from: https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb

num_tokens() IS OFTEN USED IN KNOWLEDGE.PY (and other files??) WITHOUT SPECIFYING THE EMBEDDING MODEL.

In [1]:
# !pip install datasets, sentencepiece, transformers, accelerate, tiktoken, rouge_score, evaluate rouge_score bleu_score
import sys

import openai.error

sys.path.append("modules")
from modules.config import *
from modules.knowledge import *
from modules.chatbot import *
from modules.embedding_functions import *
from modules.data_extraction import *
from modules.data_preprocessing import *
from modules.gpt_ans_extraction import *
from modules.query import *
sys.path.remove("modules")

No GPU available, using a CPU


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\point\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\point\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\point\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\point\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\point\anaconda3\lib\site-packages\transformers\convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have

No GPU available, using a CPU


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\point\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\point\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\point\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\point\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### NQ Data Extraction and Preprocessing

In [ ]:
### Data extraction
training = AllData(cache_dir='/content/drive/MyDrive/Diss/Datasets', default='dev') # NEED TO CORRECT THIS (train/test separation for efficiency)
training.export_simplified_dataset(path="/content/drive/MyDrive/Diss/Output/simplified_dataset_validation_new.csv")

### Data preprocessing
training_data = TrainingData(save_dir=f'{OUTPUT_DIR}/all_data_{short_model_name}')

In [2]:
# Load the dataset and extract only the answerable questions
# output_dir = 'G:\My Drive\Diss\Output'
# all_data = load_from_disk(f'{OUTPUT_DIR}/all_data_{short_model_name}').shuffle(seed=9)
all_ans_data = training_data.training_data.filter(lambda row: (row["answer"] != NO_ANS_TOKEN))
all_ans_data

Filter:   0%|          | 0/16433 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4109 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'url', 'content', 'question', 'answer', 'start_token', 'end_token', 'length', 'input_ids', 'attention_mask', 'labels', 'num_tokens'],
        num_rows: 11504
    })
    test: Dataset({
        features: ['id', 'title', 'url', 'content', 'question', 'answer', 'start_token', 'end_token', 'length', 'input_ids', 'attention_mask', 'labels', 'num_tokens'],
        num_rows: 2876
    })
})

In [ ]:
# Iterate through the dataset and query GPT to extract a natural-language answer
splits = ['train', 'test']
for s in splits:
    all_timestamps = []
    responses = []
    # Obtain GPT answers/responses
    for i in tqdm(range(len(all_ans_data[s]))): # Due to the RPM
        all_timestamps = pause_if_needed(all_timestamps)

        formatted_text = format_request(all_ans_data[s][i])
        inputs = [
                {"role": "system", "content": f"You answer questions by only using a provided context."},
                {"role": "user", "content": formatted_text},
            ]
        response, all_timestamps = query_gpt(inputs, all_timestamps)
        responses.append(response.choices[0].message.content)

    # Export to txt file
    file_name = f"{s}_all.txt"
    # Open the file in write mode and save the list to the file
    with open(file_name, "w", encoding="utf-8") as file:
        for item in responses:
            file.write(item + "\n")

    # Export as HF dataset
    df_pandas = all_ans_data[s].to_pandas()
    df_pandas['gpt_ans'] = responses
    new_dataset = Dataset.from_pandas(df_pandas)
    new_dataset.save_to_disk(f"{OUTPUT_DIR}/{short_model_name}_{s}_split")

  0%|          | 45/11504 [00:56<3:53:04,  1.22s/it]

An exception occurred: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Mon, 24 Jul 2023 13:14:26 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7ebc65811d5535da-LHR', 'alt-svc': 'h3=":443"; ma=86400'}


  0%|          | 54/11504 [06:27<23:50:21,  7.50s/it] 

Major error The server is overloaded or not ready yet.


  1%|          | 71/11504 [07:27<3:23:18,  1.07s/it] 

...

 15%|█▍        | 1722/11504 [1:12:16<14:24:08,  5.30s/it] 

Major error The server is overloaded or not ready yet.


 15%|█▌        | 1744/11504 [1:13:21<2:42:55,  1.00s/it] 

Major error The server is overloaded or not ready yet.


 16%|█▌        | 1798/11504 [1:15:02<2:29:11,  1.08it/s] 

In [ ]:
# Now the new (GPT) answers can be merged with the original dataset
# Load the original data and GPT data generated above
train_dataset_pandas = training_data.training_data['train'].to_pandas()
test_dataset_pandas = training_data.training_data['test'].to_pandas()
# updated_train_dataset_pandas = load_from_disk(f"assets/{short_model_name}_train_split").to_pandas()
# updated_test_dataset_pandas = load_from_disk(f"assets/{short_model_name}_test_split").to_pandas()
updated_train_dataset_pandas = load_from_disk(f"{OUTPUT_DIR}/{short_model_name}_train_split").to_pandas()
updated_test_dataset_pandas = load_from_disk(f"{OUTPUT_DIR}/{short_model_name}_test_split").to_pandas()

# Merge on id and keep all rows
train_merged = pd.merge(train_dataset_pandas, updated_train_dataset_pandas[['id', 'gpt_ans']], on='id', how='outer')
test_merged = pd.merge(test_dataset_pandas, updated_test_dataset_pandas[['id', 'gpt_ans']], on='id', how='outer')

In [ ]:
# Remove any examples where it apparently had an answer but GPT couldn't extract oen
train_merged = train_merged[train_merged['gpt_ans'] != NO_ANS_TOKEN]
test_merged = test_merged[test_merged['gpt_ans'] !=  NO_ANS_TOKEN]

# Update the answers to match the GPT ones
train_merged.loc[train_merged['answer'] != NO_ANS_TOKEN, 'answer'] = train_merged.loc[train_merged['answer'] != NO_ANS_TOKEN, 'gpt_ans']
test_merged.loc[test_merged['answer'] != NO_ANS_TOKEN, 'answer'] = test_merged.loc[test_merged['answer'] != NO_ANS_TOKEN, 'gpt_ans']

merged_dataset = DatasetDict({
        "train": Dataset.from_pandas(train_merged),
        "test": Dataset.from_pandas(test_merged),
    })

# Maintain the answerable/non-answerable balance
def ensure_ans_non_ans_balance(dataset, dataset_splits=('training', 'validation'), proportion=0.3, seed=SEED):
    # Extracting the unanswerable examples
    no_ans = dataset.filter(lambda row: (row["answer"] == NO_ANS_TOKEN))
    good_ans = dataset.filter(lambda row: (row["answer"] != NO_ANS_TOKEN))

    # Discarding some unanswerable examples so the answer-no_ans ratio is favourable in each split
    processed_datasets_dict = {}
    for split in dataset_splits:
      num_no_ans = proportion*len(good_ans[split])/(1-proportion)
      no_ans_keep = no_ans[split].train_test_split(train_size=num_no_ans/len(no_ans[split]), seed=seed)['train']
      processed_datasets_dict[split] = concatenate_datasets([no_ans_keep, good_ans[split]])

    processed_dataset = DatasetDict({
            dataset_splits[0]: processed_datasets_dict[dataset_splits[0]],
            dataset_splits[1]: processed_datasets_dict[dataset_splits[1]],
        })
    shuffled_dataset = processed_dataset.shuffle(seed=seed)
    return shuffled_dataset

final_dataset = ensure_ans_non_ans_balance(merged_dataset, dataset_splits=['train', 'test'])

# As can be seen, the 80/20 train-test split has been maintained
len(final_dataset['train'])/(len(final_dataset['train'])+len(final_dataset['test']))

In [ ]:
# Finally, the questions/contexts and answers need to be tokenised to override previous values
def tokenise(data):
    # tokenize the inputs (questions and contexts)
    additional_cols = tokeniser(data['content'], data['question'], truncation=False)

    # tokenize the answers
    targets = tokeniser(text_target=data['answer'], truncation=False)

    #set labels
    additional_cols['labels'] = targets['input_ids']
    additional_cols['num_tokens'] = [len(row) for row in additional_cols["input_ids"]]
    return additional_cols

final_dataset = final_dataset.map(tokenise, batched = True)
final_dataset.save_to_disk(f'{OUTPUT_DIR}/all_data_{short_model_name}_gpt_updated')

# NQ Model Training

In [ ]:
# Load data
all_data = load_from_disk(f'{OUTPUT_DIR}/all_data_{short_model_name}_gpt_updated') # add or remove the suffix a required

# Connect to HuggingFace
HfFolder.save_token("ADD_TOKEN_HERE")
!git config --global user.email "pointon.joel@gmail.com"
!git config --global user.name "Joel Pointon"

# Training arguments/config
batch_size = 8 # 64
num_train_epochs = 8
logging_steps = len(all_data["training"]) // batch_size # Show the training loss with every epoch
args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}_updated",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    push_to_hub=True,
    seed=9,
    # optim="adafactor"
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokeniser.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokeniser.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokeniser.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    # result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    result = {key: value * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

data_collator = DataCollatorForSeq2Seq(tokeniser, model=model)

def model_init():
  return AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map="auto", torch_dtype="auto")

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=all_data["train"],
    eval_dataset=all_data["test"],
    data_collator=data_collator,
    tokenizer=tokeniser,
    compute_metrics=compute_metrics,
)

path = f"/content/drive/MyDrive/Diss/Output/{short_model_name}-finetuned-natural-questions"
trainer.train()
trainer.push_to_hub()
trainer.save_model(path)

In [9]:
# NEED TO FIX THIS
filename = 'assets/knowledge/Fundamentals_of_Digital_Image_Processing_Textbook.pdf'
doc = fitz.open(filename)
page_limit = None
all_text = ''
# Iterate through the content
for page in doc:
    page_limit = doc.page_count if not page_limit else page_limit
    if page.number <= page_limit:
        block_content = page.get_text("blocks") #.encode("utf8") # "blocks"
        for block in block_content:
            if block[6] == 0:  # I.e. only extract text
                plain_text = unidecode(block[4])  # .decode('latin1') #.decode('utf-8')
                all_text += plain_text
    else:
        pass

In [2]:
# get_embedding("Joel", embedding_model=self.embedding_model)
# filename = 'assets/knowledge/Fundamentals_of_Digital_Image_Processing_Textbook.pdf'
filename = 'assets/knowledge/Fundamentals_of_Digital_Image_Processing_Textbook.pdf'
CompVisionKnowledge = Knowledge('CompVisionPDF', 'GPT')
# for page in WIKI_PAGES:
# CompVisionKnowledge.append_wikipedia_page(WIKI_PAGE) # page
# CompVisionKnowledge.append_pdf(filename, 'CompVisionPDF')
CompVisionKnowledge.append_pdf(filename, 'CompVisionPDF')
# CompVisionKnowledge.export_to_csv(GPT_KNOWLEDGE_FILENAME)
CompVisionKnowledge.df

RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-fD9aTSHfvzCKrinOV5h9G0Xm on tokens per min. Limit: 150000 / min. Current: 0 / min. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.

### MLM Training

In [8]:
# Extracting training content
all_text = ''.join(CompVisionKnowledge.df['Content'].tolist())
with open(f'assets/knowledge/Digital_Image_Processing_Textbook.txt', "w") as f:
    f.write(all_text)

In [2]:
CompVisionKnowledgeBERT = Knowledge(WIKI_PAGE, 'BERT')
CompVisionKnowledgeBERT.append_wikipedia_page(WIKI_PAGE)
# save document chunks and embeddings
CompVisionKnowledgeBERT.export_to_csv(BERT_KNOWLEDGE_FILENAME)
CompVisionKnowledgeBERT.df


KeyboardInterrupt



# Search
Now we'll define a search function that:

Takes a user query and a dataframe with text & embedding columns
Embeds the user query with the OpenAI API
Uses distance between query embedding and text embeddings to rank the texts
Returns two lists:
The top N texts, ranked by relevance
Their corresponding relevance scores

In [8]:
# Todo:
# I need to make it more efficient on the number of tokens
# Check num_tokens usage and specifying the encoding model
# Fix the potential issue fo GPT sections being longer than 1024 tokens when using BART

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.

KeyboardInterrupt



In [13]:
CompVisionGPT = ChatBot("Computer Vision", 'assets/' + GPT_KNOWLEDGE_FILENAME)
print(Query.ask('What is PCA?', CompVisionGPT, show_source=True))

C:\Users\point\OneDrive\Documents\Standard Files\University\Masters\Diss\MSc-Diss\code\modules\query.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.knowledge_used['Index'] = np.arange(len(self.knowledge_used)) + 1
C:\Users\point\OneDrive\Documents\Standard Files\University\Masters\Diss\MSc-Diss\code\modules\query.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.knowledge_used['Cumulative_tokens'] = self.knowledge_used['Tokens'].cumsum()
C:\Users\point\OneDrive\Documents\Standard Files\Un

PCA stands for Principal Component Analysis, which is a statistical technique used for dimensionality reduction of correlated data. It involves finding a new set of uncorrelated variables (or axes) that are linear combinations of the original variables, and which capture the maximum amount of variance in the data. PCA is commonly used in digital imaging for tasks such as data compression, automated facial recognition, and determination of the orientation of basic shapes.

To construct this answer, I used the following documents : 

1. CompVisionPDF-> p.265/266/267:
The reader is probably familiar with the common saying that goes something along the
lines of 'Why ...

2. CompVisionPDF-> p.268/269/270:
For a 2-D space, we must stop here (there are no more dimensions left). Figure 9.9
shows our exampl...

Total tokens used: 3227


C:\Users\point\OneDrive\Documents\Standard Files\University\Masters\Diss\MSc-Diss\code\modules\query.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.knowledge_used['Output'] = '\n\n' + self.knowledge_used['Index'].astype(str) + '. ' + self.knowledge_used[


In [13]:
CompVisionBERT = ChatBot("Computer Vision", 'assets/' + BERT_KNOWLEDGE_FILENAME)
print(Query.ask_bert('When did universities begin teaching Computer Vision?', CompVisionBERT))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ValueError: operands could not be broadcast together with shapes (768,) (1536,) 

In [20]:
CompVisionGPT = ChatBot("Computer Vision", 'assets/' + GPT_KNOWLEDGE_FILENAME)
print(Query.ask_bart('When did Universities begin teaching Computer Vision?', CompVisionGPT, show_source=True))  # What if the GPT knowledge sections are longer than 1024 tokens?? Need to account for this!

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices